# IML - Model exploration

Model has to be a Convolutional Neural Network, consuming as input a spectrogram and performing a
classification task.

In [10]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.api.utils import image_dataset_from_directory
import keras
from keras.api.layers import Dense
from keras.api import Model
from keras.api.optimizers import SGD
from keras.api.callbacks import EarlyStopping
from datetime import datetime
from sklearn.utils.class_weight import compute_class_weight

### 1. Loading dataset

Use convenient function image_dataset_from_directory from keras. Handles train/validation split, 
shuffle, converting to grayscale etc.


In [2]:
image_size = (256, 256)
batch_size = 32

data_path = os.path.join(os.getcwd(), "..", "data", "overlap_000_res_256x256_train_test_split")
train_path = os.path.join(data_path, "train")
train_ds, valid_ds = image_dataset_from_directory(train_path, color_mode="grayscale", seed=42,
                                                   image_size=image_size, batch_size=batch_size,
                                                   subset="both", validation_split=0.2)

train_ds = train_ds.map(lambda X, y: (tf.cast(X, tf.float32), tf.cast(y, tf.float32)))
train_ds: tf.data.Dataset = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.map(lambda X, y: (tf.cast(X, tf.float32), tf.cast(y, tf.float32)))
valid_ds: tf.data.Dataset = valid_ds.prefetch(tf.data.AUTOTUNE)

Found 63840 files belonging to 2 classes.
Using 51072 files for training.
Using 12768 files for validation.


I0000 00:00:1731506205.275213   16772 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


### 2. Model

Time for fun :)

#### 1. Simple neural network

In [3]:
LOG_DIR = os.path.join(os.getcwd(), "..", "logs")
FIT_LOG_DIR = os.path.join(LOG_DIR, "fit")


def train_model(train_ds, valid_ds, model_constructor, batch_size: int, epochs: int,
                optimizer, loss, log_dir: str = FIT_LOG_DIR):
    model = model_constructor()
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

    model_log_dir = os.path.join(log_dir, datetime.now().strftime(f"%Y%m%d-%H%M%S-{model.name}"))
    tensorboard_cb = keras.callbacks.TensorBoard(log_dir=model_log_dir, histogram_freq=1,
                                                    write_images=True)
    history = model.fit(x=train_ds, validation_data=valid_ds,batch_size=batch_size, epochs=epochs,
                callbacks=[tensorboard_cb])
    return model, history

In [4]:
def simple_nn() -> Model:
    input_shape = (256, 256, 1)
    inputs = keras.Input(input_shape, dtype=np.float32)
    x = keras.layers.Rescaling(scale=1./255)(inputs)
    x = keras.layers.Flatten("channels_last")(x)
    x = Dense(256, activation="relu")(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs, name="SimpleNN")
    return model


model = simple_nn()
model.summary()

Model: "SimpleNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    16,777,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,777,729 (64.00 MB)

 Trainable params: 16,777,729 (64.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_model(train_ds, valid_ds, simple_nn, batch_size, 20, optimizer="sgd", loss="binary_crossentropy")

Epoch 1/20


I0000 00:00:1731257654.304787   12587 service.cc:148] XLA service 0x7f6418007680 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731257654.305341   12587 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-11-10 17:54:14.377465: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731257654.563706   12587 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-10 17:54:15.732436: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 188 bytes spill stores, 188 bytes spill loads

2024-11-10 17:54:15.851370: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28_0

  17/1288 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6468 - loss: 1.0842 

I0000 00:00:1731257656.585596   12587 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1288/1288 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6970 - loss: 0.6123

2024-11-10 17:54:39.378682: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_24_0', 232 bytes spill stores, 232 bytes spill loads



1288/1288 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - accuracy: 0.6970 - loss: 0.6122 - val_accuracy: 0.7275 - val_loss: 0.5687
Epoch 2/20
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 26s 20ms/step - accuracy: 0.7182 - loss: 0.5269 - val_accuracy: 0.7825 - val_loss: 0.4108
Epoch 3/20
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.7446 - loss: 0.4920 - val_accuracy: 0.8421 - val_loss: 0.3820
Epoch 4/20
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.7844 - loss: 0.4373 - val_accuracy: 0.8189 - val_loss: 0.3498
Epoch 5/20
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 25s 19ms/step - accuracy: 0.8063 - loss: 0.3995 - val_accuracy: 0.8592 - val_loss: 0.3706
Epoch 6/20
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 25s 19ms/step - accuracy: 0.8297 - loss: 0.3672 - val_accuracy: 0.8484 - val_loss: 0.3082
Epoch 7/20
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.8409 - loss: 0.3481 - val_accuracy: 0.8875 - val_loss: 0.2813
Epoch 8/20
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 25s 19ms/step - accuracy: 0.8500 - loss: 0.32

<Functional name=SimpleNN, built=True>

Those results are really surprising, we still haven't reached overfitting. Let's prolong our training
to 50 epochs and see when model overfitts. 

Still no need for tweaking optimizer or loss metaparameters.

In [8]:
model_50_epochs = train_model(train_ds, valid_ds, simple_nn, batch_size, 50,
                              optimizer="sgd", loss="binary_crossentropy")

Epoch 1/50


I0000 00:00:1731259377.689293   24084 service.cc:148] XLA service 0x7f4168006560 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731259377.689395   24084 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-11-10 18:22:57.742256: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731259377.877004   24084 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-10 18:22:59.025333: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 188 bytes spill stores, 188 bytes spill loads

2024-11-10 18:22:59.044234: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28_0

  27/1288 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - accuracy: 0.6698 - loss: 1.1676

I0000 00:00:1731259379.804535   24084 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1283/1288 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6973 - loss: 0.6218

2024-11-10 18:23:24.288150: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_24_0', 232 bytes spill stores, 232 bytes spill loads



1288/1288 ━━━━━━━━━━━━━━━━━━━━ 30s 21ms/step - accuracy: 0.6973 - loss: 0.6216 - val_accuracy: 0.7737 - val_loss: 0.5329
Epoch 2/50
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.7194 - loss: 0.5302 - val_accuracy: 0.7112 - val_loss: 0.4726
Epoch 3/50
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.7534 - loss: 0.4811 - val_accuracy: 0.8473 - val_loss: 0.3753
Epoch 4/50
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 25s 19ms/step - accuracy: 0.7881 - loss: 0.4333 - val_accuracy: 0.8114 - val_loss: 0.3590
Epoch 5/50
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - accuracy: 0.8081 - loss: 0.3976 - val_accuracy: 0.8232 - val_loss: 0.3955
Epoch 6/50
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 25s 19ms/step - accuracy: 0.8316 - loss: 0.3656 - val_accuracy: 0.8317 - val_loss: 0.3240
Epoch 7/50
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.8404 - loss: 0.3469 - val_accuracy: 0.8896 - val_loss: 0.2678
Epoch 8/50
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 22s 17ms/step - accuracy: 0.8516 - loss: 0.32

What we do **have to** take into account is that our classes are imbalanced - 30/70 ratio of 
class 1 to class 0. Let's try weighting loss for particular classes.

In [5]:
y_train = np.concatenate([y for X, y in train_ds.as_numpy_iterator()])
class_weights = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weights = {0: class_weights[0], 1: class_weights[1]}
class_weights

2024-11-13 14:57:09.749390: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


{0: np.float64(0.7244666364049024), 1: np.float64(1.6137512639029323)}

In [20]:
model = simple_nn()
model.name = "SimpleNN_class_weights"
model.compile(optimizer="sgd", loss="binary_crossentropy", metrics=["accuracy"])

model_log_dir = os.path.join(FIT_LOG_DIR, datetime.now().strftime(f"%Y%m%d-%H%M%S-{model.name}"))
tensorboard_cb = keras.callbacks.TensorBoard(log_dir=model_log_dir, histogram_freq=1,
                                                write_images=True)

history = model.fit(x=train_ds, validation_data=valid_ds, batch_size=batch_size, epochs=50,
                    class_weight=class_weights, callbacks=[tensorboard_cb])

Epoch 1/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.5963 - loss: 0.7021 - val_accuracy: 0.7650 - val_loss: 0.5280
Epoch 2/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.6877 - loss: 0.5969 - val_accuracy: 0.7863 - val_loss: 0.4358
Epoch 3/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7340 - loss: 0.5401 - val_accuracy: 0.6280 - val_loss: 0.6962
Epoch 4/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.7681 - loss: 0.4863 - val_accuracy: 0.8602 - val_loss: 0.3119
Epoch 5/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7990 - loss: 0.4309 - val_accuracy: 0.4704 - val_loss: 1.4446
Epoch 6/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8205 - loss: 0.3916 - val_accuracy: 0.8672 - val_loss: 0.2782
Epoch 7/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8364 - loss: 0.3596 - val_accuracy: 0.8568 - val_loss: 0.2940
Epoch 8/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.8351 -

Class weights outtakes: in this very take, it has improved validation loss and minimally increased
training loss. We can conclude that balancing the data improved generalization.

### Optimizer parameters
#### 1. Learning rate
From the previous experiment we can tell that learning rate was too high - around 50th epoch we could
have observed potential overfitting. The plan is to:

1. Develop a few models with different constant learning rates
2. Develope a few models with exponential decay
3. Compare results and pick the best model

***Constant learning rates:***

In [7]:
learning_rates = np.logspace(-2, 0, 5)
lr_dir = os.path.join(LOG_DIR, "learning_rate")

loss = "binary_crossentropy"
metrics = ["accuracy"]
weighted_metrics = ["accuracy"]

for learning_rate in learning_rates:
    optimizer = SGD(learning_rate=learning_rate)

    model = simple_nn()
    model.name = f"simplenn_lr_{learning_rate}"
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics, weighted_metrics=weighted_metrics)

    model_log_dir = os.path.join(lr_dir, datetime.now().strftime(f"%Y%m%d-%H%M%S-{model.name}"))
    tensorboard_cb = keras.callbacks.TensorBoard(log_dir=model_log_dir, histogram_freq=1,
                                                    write_images=True)

    history = model.fit(x=train_ds, validation_data=valid_ds, batch_size=batch_size, epochs=50,
                        class_weight=class_weights, callbacks=[tensorboard_cb])

Epoch 1/50


I0000 00:00:1731506321.504975   16989 service.cc:148] XLA service 0x7f05240065d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731506321.508018   16989 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-11-13 14:58:41.538707: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731506321.667850   16989 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-13 14:58:42.893160: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 188 bytes spill stores, 188 bytes spill loads

2024-11-13 14:58:43.081935: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36_0

  18/1596 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.4962 - loss: 1.7616 - weighted_accuracy: 0.4721  

I0000 00:00:1731506324.127316   16989 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1596/1596 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.5997 - loss: 0.7025 - weighted_accuracy: 0.5766 - val_accuracy: 0.4427 - val_loss: 0.7933 - val_weighted_accuracy: 0.4427
Epoch 2/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.6774 - loss: 0.6118 - weighted_accuracy: 0.6722 - val_accuracy: 0.7151 - val_loss: 0.6126 - val_weighted_accuracy: 0.7151
Epoch 3/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.7326 - loss: 0.5451 - weighted_accuracy: 0.7332 - val_accuracy: 0.8432 - val_loss: 0.4092 - val_weighted_accuracy: 0.8432
Epoch 4/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.7732 - loss: 0.4711 - weighted_accuracy: 0.7745 - val_accuracy: 0.8518 - val_loss: 0.3524 - val_weighted_accuracy: 0.8518
Epoch 5/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.7990 - loss: 0.4288 - weighted_accuracy: 0.8015 - val_accuracy: 0.8300 - val_loss: 0.3303 - val_weighted_accuracy: 0.8300
Epoch 6/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/

2024-11-13 16:19:40.292700: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 8388736 bytes after encountering the first element of size 8388736 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-13 16:19:40.293671: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 8388992 bytes after encountering the first element of size 8388992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


1596/1596 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.5113 - loss: 0.6947 - weighted_accuracy: 0.4970 - val_accuracy: 0.6933 - val_loss: 0.6775 - val_weighted_accuracy: 0.6933
Epoch 12/50
  16/1596 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.4860 - loss: 0.7087 - weighted_accuracy: 0.4417

2024-11-13 16:20:01.269720: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 8388736 bytes after encountering the first element of size 8388736 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-13 16:20:01.270064: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 8388992 bytes after encountering the first element of size 8388992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


1596/1596 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.5050 - loss: 0.6950 - weighted_accuracy: 0.4913 - val_accuracy: 0.6933 - val_loss: 0.6796 - val_weighted_accuracy: 0.6933
Epoch 13/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.5179 - loss: 0.6947 - weighted_accuracy: 0.4955 - val_accuracy: 0.3067 - val_loss: 0.7107 - val_weighted_accuracy: 0.3067
Epoch 14/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.5096 - loss: 0.6945 - weighted_accuracy: 0.4984 - val_accuracy: 0.6933 - val_loss: 0.6855 - val_weighted_accuracy: 0.6933
Epoch 15/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.5061 - loss: 0.6946 - weighted_accuracy: 0.4991 - val_accuracy: 0.3067 - val_loss: 0.6984 - val_weighted_accuracy: 0.3067
Epoch 16/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 23s 14ms/step - accuracy: 0.4976 - loss: 0.6946 - weighted_accuracy: 0.4938 - val_accuracy: 0.6933 - val_loss: 0.6508 - val_weighted_accuracy: 0.6933
Epoch 17/50
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 19s 

Looking at the graphs, all learning rates above 0.033 do not converge, let's look at smaller 
learning rates. 

To shorten the fitting time, let's drop from 50 epochs to 30 and implement early 
stopping with parience of 8 epochs, starting after 8 epochs and restoring the best set of weights.

In [14]:
learning_rates = np.logspace(-5, -1, 9)
lr_dir = os.path.join(LOG_DIR, "learning_rate")

loss = "binary_crossentropy"
metrics = ["accuracy"]
weighted_metrics = ["accuracy"]
early_stopping = EarlyStopping(patience=8, start_from_epoch=8, restore_best_weights=True, verbose=1)

models = [{model: None, history: None} for _ in learning_rates]

for i, learning_rate in enumerate(learning_rates):
    optimizer = SGD(learning_rate=learning_rate)

    model = simple_nn()
    model.name = f"simplenn_lr_{learning_rate}"
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics, weighted_metrics=weighted_metrics)

    model_log_dir = os.path.join(lr_dir, datetime.now().strftime(f"%Y%m%d-%H%M%S-{model.name}"))
    tensorboard_cb = keras.callbacks.TensorBoard(log_dir=model_log_dir, histogram_freq=1,
                                                    write_images=True)

    history = model.fit(x=train_ds, validation_data=valid_ds, batch_size=batch_size, epochs=30,
                        class_weight=class_weights, callbacks=[tensorboard_cb, early_stopping])
    models[i]["model"] = model
    models[i]["history"] = history


Epoch 1/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.5726 - loss: 0.6955 - weighted_accuracy: 0.5206 - val_accuracy: 0.6113 - val_loss: 0.6648 - val_weighted_accuracy: 0.6113
Epoch 2/30
   3/1596 ━━━━━━━━━━━━━━━━━━━━ 1:24 53ms/step - accuracy: 0.6042 - loss: 0.6767 - weighted_accuracy: 0.5520

2024-11-13 17:39:00.415464: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 8388992 bytes after encountering the first element of size 8388992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


1596/1596 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - accuracy: 0.6137 - loss: 0.6718 - weighted_accuracy: 0.5889 - val_accuracy: 0.6205 - val_loss: 0.6572 - val_weighted_accuracy: 0.6205
Epoch 3/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - accuracy: 0.6289 - loss: 0.6603 - weighted_accuracy: 0.6124 - val_accuracy: 0.6378 - val_loss: 0.6469 - val_weighted_accuracy: 0.6378
Epoch 4/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 25s 15ms/step - accuracy: 0.6418 - loss: 0.6519 - weighted_accuracy: 0.6284 - val_accuracy: 0.6484 - val_loss: 0.6395 - val_weighted_accuracy: 0.6484
Epoch 5/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.6519 - loss: 0.6445 - weighted_accuracy: 0.6410 - val_accuracy: 0.6523 - val_loss: 0.6360 - val_weighted_accuracy: 0.6523
Epoch 6/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - accuracy: 0.6633 - loss: 0.6381 - weighted_accuracy: 0.6541 - val_accuracy: 0.6660 - val_loss: 0.6269 - val_weighted_accuracy: 0.6660
Epoch 7/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/

2024-11-13 18:46:16.879739: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 8388736 bytes after encountering the first element of size 8388736 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2024-11-13 18:46:16.880122: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 8388992 bytes after encountering the first element of size 8388992 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


1596/1596 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.4418 - loss: 0.6908 - weighted_accuracy: 0.5168 - val_accuracy: 0.3078 - val_loss: 0.6962 - val_weighted_accuracy: 0.3078
Epoch 13/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.5271 - loss: 0.6919 - weighted_accuracy: 0.4969 - val_accuracy: 0.3084 - val_loss: 0.6950 - val_weighted_accuracy: 0.3084
Epoch 14/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.5156 - loss: 0.6918 - weighted_accuracy: 0.5038 - val_accuracy: 0.3089 - val_loss: 0.6935 - val_weighted_accuracy: 0.3089
Epoch 15/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.5729 - loss: 0.6917 - weighted_accuracy: 0.4973 - val_accuracy: 0.3090 - val_loss: 0.6959 - val_weighted_accuracy: 0.3090
Epoch 16/30
1596/1596 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - accuracy: 0.5429 - loss: 0.6918 - weighted_accuracy: 0.4982 - val_accuracy: 0.3094 - val_loss: 0.6963 - val_weighted_accuracy: 0.3094
Epoch 16: early stopping
Restoring model weight

### Sandbox

In [2]:
np.repeat(5, 10)

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [2]:
bytes = 128 * 256 * 4 * 64369
mb = bytes / 1000 / 1000
mb

8436.973568

In [17]:
np.float128(0.8)*np.float128(0.8)

np.longdouble('0.64000000000000007106')